In [ ]:
import numpy as np

data_aug = np.load('D:\dev-project\DiffusionGenerate-tr-nosp1\generative_model\data_processed\EEG Eye State/augment\ddpm_fake_1_eeg-0.2-diff.npy')
print(data_aug.shape)
data_raw = np.load('D:\dev-project\DiffusionGenerate-tr-nosp1\generative_model\data_processed\EEG Eye State\processed_small/train-0.2.npz')
data_1 = data_raw['data']
target_1 = data_raw['target']
print(data_1.shape)
print(target_1.shape)

combined_data = np.vstack([data_aug, data_1])
target1 = np.ones(data_aug.shape[0]).astype(np.int64)
combined_target = np.concatenate([target1, target_1])

print(combined_data.shape)
print(combined_target.shape)

np.savez_compressed("D:\dev-project\DiffusionGenerate-tr-nosp1\generative_model\data_processed\EEG Eye State/augment/train.npz", data=combined_data, target=combined_target)


In [ ]:
import numpy as np

# Cargar el dataset de entrenamiento
train_data = np.load('D:\dev-project\DiffusionGenerate-tr-nosp1\generative_model\data_processed\EEG Eye State/augment/train.npz')
# train_data = np.load('D:\dev-project\DiffusionGenerate-tr-nosp1\generative_model\data_processed\EEG Eye State\processed_small/train.npz')
X_train = train_data['target']
y_train = train_data['data']

print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)

In [ ]:
# 获取所有类别及其样本数
unique_classes = np.unique(X_train)
num_classes = len(unique_classes)
print("类别总数:", num_classes)
print("类别列表:", unique_classes)

# 统计每个类别的样本数量
class_counts = {class_label: np.sum(X_train == class_label) for class_label in unique_classes}
print("每个类别的样本数:", class_counts)


In [ ]:
from sklearn.preprocessing import StandardScaler

# Reshape the data to 2D for scaling
y_train_reshaped = y_train.reshape(y_train.shape[0], -1)

# Fit and transform the training data
scaler = StandardScaler()
y_train_scaled = scaler.fit_transform(y_train_reshaped)

# Reshape back to original shape
y_train_scaled = y_train_scaled.reshape(y_train.shape)

print("Shape of y_train_scaled:", y_train_scaled.shape)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input

# Asumimos que y_train_scaled es nuestro conjunto de entrenamiento
# y que X_train contiene las etiquetas

# Definir la forma de entrada
input_shape = (y_train_scaled.shape[1], y_train_scaled.shape[2])

model = Sequential([
    Input(shape=input_shape),
    LSTM(64, return_sequences=True),
    Dropout(0.5),
    LSTM(64, return_sequences=False),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
import numpy as np

# 加载原始测试数据
data_raw = np.load(r'D:\dev-project\DiffusionGenerate-tr-nosp1\generative_model\data_processed\EEG Eye State\processed_small\test.npz')
data_test = data_raw['data']
target_test = data_raw['target']


In [ ]:
from sklearn.model_selection import train_test_split

model.fit(y_train_scaled, X_train, 
          validation_data=(data_test, target_test),
          epochs=10, 
          batch_size=32)


In [ ]:
# Evaluar el modelo en el conjunto de validación balanceado
val_balanced_loss, val_balanced_accuracy = model.evaluate(data_test, target_test)
print(f"Validation Balanced Loss: {val_balanced_loss}")
print(f"Validation Balanced Accuracy: {val_balanced_accuracy}")

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Hacer predicciones
y_pred_prob = model.predict(data_test)
y_pred = (y_pred_prob > 0.5).astype(int)

# Calcular métricas adicionales
print(classification_report(target_test, y_pred))

# Matriz de confusión
cm = confusion_matrix(target_test, y_pred)
plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True, fmt='d')
plt.title('Confusion Matrix')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

#敏感性和特异性分析
tn, fp, fn, tp = confusion_matrix(target_test, y_pred, labels=[0, 1]).ravel()
sensitivity = tp / (tp + fn + 1e-9)
specificity = tn / (tn + fp + 1e-9)
print(f"sensitivity Score: {sensitivity}")
print(f"specificity Score: {specificity}")

# ROC-AUC score
auc_roc = roc_auc_score(target_test, y_pred_prob)
print(f"ROC-AUC Score: {auc_roc}")

# Curva ROC
from sklearn.metrics import roc_curve
fpr, tpr, _ = roc_curve(target_test, y_pred_prob)
plt.figure()
plt.plot(fpr, tpr, label=f'ROC curve (AUC = {auc_roc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()